# About
* Run the program (including building the database):
    * 1.1 --> 2.1 --> 3.1 --> 3.2 --> 3.3 --> 3.4 (YouTube) or 3.5 (Web Cam) --> 4.2 --> 4.3 --> 5.2 --> 5.3
* Run face recognition with pre-built database:
    * 1.1 --> 2.1 --> 4.2 --> 4.3 --> 5.2 --> 5.3
* Database building only: 
    * 1.1 --> 2.1 --> 3.1 --> 3.2 --> 3.3 --> 3.4
    
Each block is elaborated as commented.

# 1. Imports

### 1.1 Import necessary libraries and packages

In [ ]:
#import libraries and packages needed
import sys, os, dlib, glob, pickle, face_recognition, cv2, random, imutils, csv, pafy
import numpy as np
from skimage import io
from imutils import paths

# 2. Preparation

### 2.1 Create label collection: subjects

In [ ]:
#get the directories (one directory for each subject) in data folder
out_dir = "./training-data"
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
dirs = os.listdir(out_dir)

#initialize the list of known encodings and known names
knownEncodings = []
knownNames = []

#there is no label 0 in our training data so subject name for index/label 0 is empty
subject_csv_dir = "./subjects.csv"
subjects = [""]
#import subjects from existing csv file (if any) to continue the list
if os.path.exists(subject_csv_dir):
    with open('./subjects.csv') as subjects_csv:
        reader = csv.reader(subjects_csv, delimiter=",")
        subjects = list(reader)[0]
        print("[INFO] CSV file read.")

# 3. Collect Training Data

Comparison on different face detection:

https://www.learnopencv.com/face-detection-opencv-dlib-and-deep-learning-c-python/

### 3.1 relight() function

In [ ]:
#改變亮度與對比
def relight(img,alpha = 1,bias = 0):
    w, h = img.shape[:2]

    for i in range(0,w):
        for j in range(0,h):
            for c in range(3):
                tmp = int(img[i,j,c] * alpha + bias)
                if tmp > 255:
                    tmp = 255
                elif tmp < 0:
                    tmp = 0
                img[i,j,c] = tmp
    return img

### 3.2 Input specifics of data collection

In [ ]:
#input number of subject
while True:
    try:
        num_of_subject = int(input("Enter the number of subject(s): "))
    except ValueError:
        print("Try again.")
        continue
    if num_of_subject <= 0:
        print("Negative number not accepted.")
        continue
    else:
        break

#choose data capture method
while True:
    try:
        method_choice = int(input("Enter (1) YouTube or (2) Web Cam: "))
    except ValueError:
        print("Try again.")
        continue
    if method_choice != 1 and method_choice != 2:
        print("Must enter 1 or 2.")
        continue
    else:
        break

### 3.3 Import model and setups

In [ ]:
#set cropping size for the images captured
crop_size = 128

#check if there's any label/face in the collection already
collection_size = len(subjects)

# load our serialized model from disk
print("[INFO] Loading model...")
net = cv2.dnn.readNetFromCaffe("deploy.prototxt.txt", "res10_300x300_ssd_iter_140000.caffemodel")
print("[INFO] Model loaded...")

### 3.4 Capture images from YouTube video (run this if method_choice == 1)

In [ ]:
if method_choice == 1:
    print("[INFO] Capturing images...")

    #iterate through the subject
    for i in range(num_of_subject):

        #input subject name and append it to subjects
        subject_name = input("Enter name for Subject #" + str(i + collection_size) + ": ")
        start_num = 0

        #input youtube video URL
        while True:
            try:
                # set up video stream capture
                url = input("Enter YouTube URL: ")
                video = pafy.new(url)
                best = video.getbest(preftype = "mp4")
                cap = cv2.VideoCapture()
                cap.open(best.url)
                framerate = cap.get(cv2.CAP_PROP_FPS)
                video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                multipler = int(framerate * 5)
            except Exception:
                print("Try again.")
                continue
            else:
                break

        #if the input exists in subjects, no new entry will be appended
        if subject_name in subjects:
            subject_dir = out_dir + "/s" + str(subjects.index(subject_name))
            sub_dir_obj = os.fsencode(subject_dir)
            #iterate through the file names to find the correct starting index for new inputs
            if len(os.listdir(sub_dir_obj) ) == 0:
                break;
            else:
                for file in os.listdir(sub_dir_obj):
                    if file is not '':
                        file_name_str = os.fsdecode(file).split('.')
                        file_num = int(file_name_str[0])
                        start_num = max(file_num, start_num)
                start_num += 1   
        #append new entry to subjects and create respective directory 
        else:
            subjects.append(subject_name)
            #create folder for each subject (if not exist)
            subject_dir = out_dir + "/s" + str(i + collection_size)
            if not os.path.exists(subject_dir):
                os.makedirs(subject_dir)
        
        #capture images of the subject until target number reached
        n = 1
        count = 0
        print("[INFO] Capturing the face of Subject #" + str(i + collection_size) + "...")
        print("Video Length: " + str(int(video_length)))

        while(cap.isOpened()):  #check!
            #capture frame-by-frame
            if count < int(video_length):
                ret, image = cap.read()
                count += 1
                if count % multipler == 0:
                    print('Processing Image %s...' % str(n))
                    print('Frame ' + str(count))

                    #load the input image and construct an input blob for the image by resizing to a fixed 300x300 pixels and then normalizing it
                    (h, w) = image.shape[:2]
                    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))

                    #pass the blob through the network and obtain the detections and predictions
                    print("[INFO] Computing object detections...")
                    net.setInput(blob)
                    detections = net.forward()

                    #loop over the detections
                    for i in range(0, detections.shape[2]):
                        #extract the confidence (i.e., probability) associated with the prediction
                        confidence = detections[0, 0, i, 2]

                        #filter out weak detections by ensuring the 'confidence' is greater than the minimum confidence
                        if confidence > 0.95:
                            #compute the (x, y)-coordinates of the bounding box for the object
                            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                            (startX, startY, endX, endY) = box.astype("int")

                            face = image[startY : endY, startX : endX]
                            width, height, channels = face.shape
                            if min(width, height) > 128:
                                face = cv2.resize(face,(crop_size, crop_size))

                            face = relight(face,random.uniform(0.5,1.5),random.randint(-50,50))
                            cv2.imwrite(subject_dir + '/' + str(n + start_num) + '.jpg', face)
                            print('Image %s processed.' % str(n))
                            n += 1
                    if cv2.waitKey(100) & 0xFF == ord('q'):
                        break
            else:
                break
    cap.release()   
    cv2.destroyAllWindows()
    print("[INFO] Image Capture Completed.")

### 3.5 Capture images through web cam and detect faces

In [ ]:
if method_choice == 2:
    # set up web cam capture
    cap = cv2.VideoCapture(0)
    
    #input number of images per subject
    while True:
        try:
            image_per_subject = int(input("Enter the number of image per subject: "))
        except ValueError:
            print("Try again.")
            continue
        if image_per_subject <= 0:
            print("Negative number not accepted.")
        else:
            break

    print("[INFO] Capturing images...")

    #iterate through the subject
    for i in range(num_of_subject):

        #input subject name and append it to subjects
        subject_name = input("Enter name for Subject #" + str(i + collection_size) + ": ")
        start_num = 0

        #if the input exists in subjects, no new entry will be appended
        if subject_name in subjects:
            subject_dir = out_dir + "/s" + str(subjects.index(subject_name))
            sub_dir_obj = os.fsencode(subject_dir)
            #iterate through the file names to find the correct starting index for new inputs
            if len(os.listdir(sub_dir_obj) ) == 0:
                break;
            else:
                for file in os.listdir(sub_dir_obj):
                    if file is not '':
                        file_name_str = os.fsdecode(file).split('.')
                        file_num = int(file_name_str[0])
                        start_num = max(file_num, start_num)
                start_num += 1   
        #append new entry to subjects and create respective directory 
        else:
            subjects.append(subject_name)
            #create folder for each subject (if not exist)
            subject_dir = out_dir + "/s" + str(i + collection_size)
            if not os.path.exists(subject_dir):
                os.makedirs(subject_dir)

        #capture images of the subject until target number reached
        n = 1
        print("[INFO] Capturing the face of Subject #" + str(i + collection_size) + "...")
        while(cap.isOpened()):  # check !
            # capture frame-by-frame  
            if n <= image_per_subject:
                ret, image = cap.read()
                print('Processing Image %s...' % str(n))

                # load the input image and construct an input blob for the image by resizing to a fixed 300x300 pixels and then normalizing it
                (h, w) = image.shape[:2]
                blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))

                # pass the blob through the network and obtain the detections and predictions
                print("[INFO] Computing object detections...")
                net.setInput(blob)
                detections = net.forward()

                # loop over the detections
                for i in range(0, detections.shape[2]):
                    # extract the confidence (i.e., probability) associated with the prediction
                    confidence = detections[0, 0, i, 2]

                    # filter out weak detections by ensuring the 'confidence' is greater than the minimum confidence
                    if confidence > 0.95:
                        # compute the (x, y)-coordinates of the bounding box for the object
                        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                        (startX, startY, endX, endY) = box.astype("int")

                        face = image[startY : endY, startX : endX]
                        width, height, channels = face.shape
                        if min(width, height) > 128:
                            face = cv2.resize(face,(crop_size, crop_size))

                        face = relight(face,random.uniform(0.5,1.5),random.randint(-50,50))
                        cv2.imwrite(subject_dir + '/' + str(n + start_num) + '.jpg', face)
                        print('Image %s processed.' % str(n))
                        n += 1
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            else:
                break
    cap.release()   
    cv2.destroyAllWindows()
    print("[INFO] Image Capture Completed.")

# 4. Prepare Training Data & Train the Model

### 4.1 Directory Structure Tree for Training Data:
```
training-data
|-------------- s1
|               |-- 1.jpg
|               |-- ...
|               |-- k.jpg
|-------------- s2
|               |-- 1.jpg
|               |-- ...
|               |-- k.jpg
        .
        .
        .
|-------------- sN
|               |-- 1.jpg
|               |-- ...
|               |-- k.jpg
```

### 4.2 prepare_training_data() function

In [ ]:
def prepare_training_data(data_folder_path):
    
    #get the directories (one directory for each subject) in data folder
    dirs = os.listdir(data_folder_path)
    
    #iterate through each subject's respective folder
    for dir_name in dirs:

        #ignore any directory not starting with 's'
        if not dir_name.startswith("s"):
            continue;

        #extract label number by removing 's' in dir_name
        label = int(dir_name.replace("s", ""))
        
        print("Training Subject #" + str(label) + ": ")

        #build path of directory containin images for current subject subject
        #e.g. "training-data/s1"
        subject_dir_path = data_folder_path + "/" + dir_name

        #get the images names that are inside the given subject directory
        subject_images_names = os.listdir(subject_dir_path)

        for image_name in subject_images_names:
            print("s" + str(label) + "/" + image_name + " completed.")
            
            #create path to the image
            image_path = subject_dir_path + "/" + image_name
            
            # load the input image and convert it from BGR (OpenCV ordering)
            # to dlib ordering (RGB)
            image = cv2.imread(image_path)
            if image is None:
                continue;
            rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # detect the (x, y)-coordinates of the bounding boxes
            # corresponding to each face in the input image
            boxes = face_recognition.face_locations(rgb, model = "cnn")

            # compute the facial embedding for the face
            encodings = face_recognition.face_encodings(rgb, boxes)

            # loop over the encodings
            for encoding in encodings:
                # add each encoding + name to our set of known names and
                # encodings
                knownEncodings.append(encoding)
                knownNames.append(subjects[label])
    
    data = {"encodings": knownEncodings, "names": knownNames}
    
    return data

### 4.3 Prepare training data and train the model

In [ ]:
print("[INFO] Training model...")
data = prepare_training_data("./training-data")
print("[INFO] Model Training Completed.")

print("Total faces: ", len(data["encodings"]))

# 5. Test

### 5.1 recognize_faces() function

In [ ]:
def recognize_faces(img_file):
    
    #load the input image and convert it from BGR to RGB
    image = cv2.imread(img_file)
    width, height, channels = image.shape
    if min(width, height) > 1280:
        image = imutils.resize(image, width = 1280)
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    #detect the (x, y)-coordinates of the bounding boxes corresponding 
    #to each face in the input image, then compute the facial embeddings for each face
    print("[INFO] Recognizing faces...")
    boxes = face_recognition.face_locations(rgb, model = "cnn")
    encodings = face_recognition.face_encodings(rgb, boxes)


    #initialize the list of names for each face detected
    names = []

    #loop over the facial embeddings
    for encoding in encodings:
        #attempt to match each face in the input image to our known encodings
        matches = face_recognition.compare_faces(data["encodings"], encoding)
        name = "Unknown"

        #check to see if we have found a match
        if True in matches:
            #find the indexes of all matched faces then initialize a dictionary 
            #to count the total number of times each face was matched
            matchedIdxs = [i for (i, b) in enumerate (matches) if b]
            counts = {}

            #loop over the matched indexes and maintain a count for each recognized face face
            for i in matchedIdxs:
                name = data["names"][i]
                counts[name] = counts.get(name, 0) + 1

            #determine the recognized face with the largest number of votes
            #(note: in the event of an unlikely tie Python will select first entry in the dictionary)
            name = max(counts, key=counts.get)

        #update the list of names
        names.append(name)
    
    #loop over the recognized faces
    for ((top, right, bottom, left), name) in zip(boxes, names):
        #draw the predicted face name on the image
        cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)
        y = top - 15 if top - 15 > 15 else top + 15
        cv2.putText(image, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        
    return image

### 5.2 Face Recognition

In [ ]:
#write subjects into csv file
with open('subjects.csv', mode = 'w') as subject_file:
    list_writer = csv.writer(subject_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    list_writer.writerow(subjects)

#write data into csv file
with open('data.csv', mode = 'w') as data_file:
    fieldnames = ["encodings", "names"]
    writer = csv.DictWriter(data_file, fieldnames = fieldnames)

    writer.writeheader()
    writer.writerow(data)
print("[INFO] CSV files written.")

#recognize test image
image = recognize_faces("test-data/test3.jpg")
print("[INFO] Recogonition Completed.")

#show the out put image 
cv2.imshow("Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()